In [6]:
from create_database import create_database
import psycopg2
from dotenv import load_dotenv
from psycopg2 import sql
import geopandas as gpd
import pandas as pd
import geoalchemy2
from sqlalchemy import create_engine
import os
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

load_dotenv()

True

In [7]:
conn_params = {
    'user': os.environ.get('POSTGRES_USER'),
    'password': os.environ.get('POSTGRES_PASSWORD', ''),
    'host': 'localhost',
    'port': os.environ.get('POSTGRES_PORT', '')
}
conn_params
# create_database(conn_params,'threatanalysis', 'postgis')

conn = psycopg2.connect(dbname='threatanalysis', **conn_params)

In [ ]:
# import psycopg2
# from psycopg2 import sql

# try:
#     # Database creation and extension setup
#     dbname = 'threatanalysis'
#     new_db_name = sql.Identifier(dbname)

#     # Wrap the connection setup in a 'with' statement
#     with psycopg2.connect(**conn_params) as conn:
#         with conn.cursor() as cursor:
#             try:
#                 # Set autocommit to True before creating the database
#                 conn.autocommit = True
#                 # Create the database if it doesn't exist
#                 cursor.execute(sql.SQL("CREATE DATABASE {}").format(new_db_name))
#                 # Reset autocommit to False after creating the database
#                 conn.autocommit = False
#                 # Switch to the new database
#                 conn = psycopg2.connect(database=dbname, **conn_params)
#                 cursor = conn.cursor()
#                 # Enable the PostGIS extension
#                 cursor.execute(sql.SQL('CREATE EXTENSION IF NOT EXISTS postgis;'))
#                 print(f"Database '{dbname}' created successfully.")
#             except psycopg2.Error as e:
#                 print(f"Error creating database '{dbname}': {e}")
# except psycopg2.Error as e:
#     print(f"Error connecting to the default database: {e}")


In [11]:
county_gdf = gpd.read_file('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json')
county_gdf.columns = county_gdf.columns.str.lower()
county_gdf = (county_gdf[['name', 'geometry']]
.copy()
.rename(columns={'name': 'event_county'})
)
county_gdf['event_county'] = county_gdf['event_county'].str.title()

In [12]:
gdf = gpd.read_file('https://raw.githubusercontent.com/nonviolent-action-lab/crowd-counting-consortium/master/ccc_compiled_2021-present.csv')

In [13]:
# gdf.head(10)
# len(gdf)
# gdf.columns
# gdf.dtypes
# # county_gdf.columns

In [14]:
gdf['lat'] = pd.to_numeric(gdf['lat'], errors='coerce')
gdf['lon'] = pd.to_numeric(gdf['lon'], errors='coerce')

from shapely.geometry import Point
geometry = [Point(lon, lat) for lon, lat in zip(gdf['lon'], gdf['lat'])]
gdf = gpd.GeoDataFrame(gdf, geometry=geometry, crs=gdf.crs)
gdf = gdf.replace('NA', None)
# # gdf = gpd.GeoDataFrame(gdf, geometry=gpd.points_from_xy(gdf.lon, gdf.lat))

# gdf[gdf.duplicated(subset='fips_code', keep=False)]
# # Convert the 'dates' column to datetime format
gdf['date'] = pd.to_datetime(gdf['date'])
gdf_2021 = gdf[gdf['date'].dt.year == 2021]

In [ ]:
sample_join = pd.join(gdf_2021, county_gdf, on='key', how='inner')


In [ ]:
sjoined_data = gpd.sjoin(gdf_2021,county_gdf, predicate='within', how='left')

In [ ]:
sjoined_data['source'] = sjoined_data.apply(lambda row: ','.join(str(val) for val in row.filter(like='source_') if val is not None), axis=1)

In [ ]:
county_gdf.head()

In [ ]:
file_path = 'Threat Analysis Project/ETL_Service/data_engineer_tech_challenge_existing_dataset.geojson'
ex_data = gpd.read_file(file_path)
# gdf.head()
# # len(gdf)
# gdf.dtypes
ex_data = ex_data.drop(columns = 'id')

In [ ]:
ex_data

In [ ]:
# ex_data.to_csv('data.csv')

In [ ]:
# ex_data[ex_data.duplicated('geometry')].sort_values(by='event_state', ascending=True)

In [ ]:
new_data = gpd.GeoDataFrame({'event_id': sjoined_data['fips_code']
                                  ,'event_date':sjoined_data['date']
                                   ,'event_state':sjoined_data['state']
                                    ,'event_county':sjoined_data['event_county']
                                     ,'event_city':sjoined_data['locality']
                                      ,'event_type':sjoined_data['type']
                                       ,'event_source':sjoined_data['source']
                                       ,'geometry':sjoined_data['geometry']})

In [ ]:
len(new_data)

In [ ]:
grouped_data = new_data.groupby(['event_id', 'event_date', 'geometry']).agg({
    'event_state': 'first',
    'event_county': 'first',
    'event_city': 'first',
    'event_type': lambda x: '; '.join(x) if any(x) else None,
    'event_source': lambda x: '; '.join(x) if any(x) else None
}).reset_index()

In [ ]:
# Define the SQL query to fetch data from the table
table_name = "eventdata"
sql_query = f"""SELECT * FROM {table_name};"""

# Use Pandas to read the query result into a DataFrame
EventData = gpd.read_postgis(sql_query, conn, geom_col='geometry')

In [ ]:
EventData = gpd.read_postgis(sql_query, conn, geom_col='geometry')

max_row_event = max(EventData['event_id'])
max_row_id = max(EventData['id'])

max_row_event = max_row_event + 1
max_row_id = max_row_id +1

In [ ]:
# Generate a new range of IDs starting from max_row
new_event_ids = range(max_row_event, max_row_event + len(grouped_data))
# Generate a new range of IDs starting from max_row
new_ids = range(max_row_id, max_row_id + len(grouped_data))

# Assign the new IDs to the 'event_id' column
grouped_data['event_id'] = new_event_ids
grouped_data['id'] = new_event_ids

grouped_data = gpd.GeoDataFrame(grouped_data)

desired_crs = 'EPSG:4326'

# Set the CRS for the GeoDataFrame
grouped_data.set_crs(desired_crs, allow_override=True)

# Display the updated grouped_data
grouped_data.head()

In [ ]:
dbname = 'threatanalysis'
engine = create_engine(f"postgresql+psycopg2://{conn_params['user']}:{conn_params['password']}@{conn_params['host']}:{conn_params['port']}/{dbname}")
engine

In [ ]:
EventData.head()

In [ ]:
desired_crs = 'EPSG:4326'

# Set the CRS for the GeoDataFrame
grouped_data = grouped_data.set_crs(desired_crs, allow_override=True)
grouped_data.to_postgis(table_name, engine, if_exists='append', index=False)

In [ ]:
grouped_data

In [ ]:
df = pd.concat([ex_data, new_data], ignore_index=True)

In [ ]:
# sjoined_data[sjoined_data.duplicated(keep=False)]

In [ ]:
# Create a GeoAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{conn_params['user']}:{conn_params['password']}@{conn_params['host']}:{conn_params['port']}/{dbname}")

# Assuming 'gdf' is your GeoPandas DataFrame
table_name = 'eventdata'  # Replace with your desired table name

# Use GeoAlchemy to create a table with geometry column
existing_data.to_postgis(table_name, engine, if_exists='replace', index=True, index_label='id')

# Close the connection
conn.close()

In [ ]:
# Create a GeoAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{conn_params['user']}:{conn_params['password']}@{conn_params['host']}:{conn_params['port']}/{dbname}")

# Assuming 'gdf' is your GeoPandas DataFrame
table_name = 'countydata'  # Replace with your desired table name

# Use GeoAlchemy to create a table with geometry column
county_gdf.to_postgis(table_name, engine, if_exists='replace', index=True, index_label='id')

# Close the connection
conn.close()

In [ ]:
# from pyspark.sql import SparkSession

# # Create a Spark session
# spark = SparkSession.builder.appName("actionlabdataretrieal").getOrCreate()

# # URL to retrieve data from
# url = "https://raw.githubusercontent.com/nonviolent-action-lab/crowd-counting-consortium/master/ccc_compiled_2021-present.csv"

# # Read data from the URL into a DataFrame
# df = spark.read.csv(url, header=True, inferSchema=True)

# # Show the DataFrame
# df.show(10)

# # Perform further data processing or analysis as needed

# # Stop the Spark session
# spark.stop()